<a href="https://colab.research.google.com/github/stemgene/Pytorch_training/blob/master/CNN%E5%9B%BE%E5%83%8F%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.bilibili.com/video/av62138405?p=4



In [53]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
print("PyTorch Version:", torch.__version__)

PyTorch Version: 1.4.0


查看MNIST数据集

In [3]:
mnist_data = datasets.MNIST(r'/gdrive/My Drive/UR/Programming/mnist_data', train=True, download=True)
mnist_data[2][0]

In [4]:
# image to tensor
mnist_data = datasets.MNIST(r'/gdrive/My Drive/UR/Programming/mnist_data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),]))
mnist_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /gdrive/My Drive/UR/Programming/mnist_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [0]:
mnist_data[2][0]

In [6]:
mnist_data[233][0].shape

torch.Size([1, 28, 28])

Normalize data

In [7]:
data = [d[0].data.cpu().numpy() for d in mnist_data]
np.mean(data)

0.13066062

首先我们定义一个基于ConvNet的简单神经网络

常见的ConvNet架构

INPUT -> [[CONV -> RELU] * N -> POOL] * M -> [FC  -> RELU] * K -> FC

In [0]:
#https://pytorch.org/docs/stable/nn.html find Conv2d
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # nn.Conv2d: (in_channels, out_channels, kernel_size, stride))
        self.conv1 = nn.Conv2d(1, 20, 5, 1) # 28 * 28 -> (28+1-5) 24 * 24
        self.conv2 = nn.Conv2d(20, 50, 5, 1) # 20 * 20
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # x: 1 * 28 * 28
        x = F.relu(self.conv1(x)) # 20 * 24 * 24
        x = F.max_pool2d(x,2,2) # kernel_size, stride. 12 * 12
        x = F.relu(self.conv2(x)) # 8 * 8
        x = F.max_pool2d(x,2,2) # 4 *4 
        x = x.view(-1, 4*4*50) # reshape (5 * 2 * 10), view(5, 20) -> (5 * 20)
        x = F.relu(self.fc1(x))
        x= self.fc2(x)
        # return x
        return F.log_softmax(x, dim=1) # log probability

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # data is an image, target is a number, represents a type

        pred = model(data) # batch_size * 10
        loss = F.nll_loss(pred, target) #cross entropy https://pytorch.org/docs/stable/nn.html?highlight=nllloss#torch.nn.NLLLoss 
        
        # SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx % 100 == 0:
            print("Train Epoch: {}, iteration: {}, Loss: {}".format(
                epoch, idx, loss.item()))

def test(model, device, test_loader):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data) # batch_size * 10, 10 means the second dimension.
            total_loss += F.nll_loss(output, target, reduction="sum").item() 
            pred = output.argmax(dim=1) # batch_size * 1, we only need the highest one, so use 'argmax()'
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    total_loss /= len(test_loader.dataset)
    acc = correct/len(test_loader.dataset) * 100.
    print("Test loss: {}, Accuracy: {}".format(total_loss, acc))

In [30]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data", train=True, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data", train=False, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)

lr = 0.01
momentum  = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

num_epochs = 2
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
    
torch.save(model.state_dict(), "mnist_cnn.pt")


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!
Train Epoch: 0, iteration: 0, Loss: 2.337495803833008
Train Epoch: 0, iteration: 100, Loss: 0.6002731323242188
Train Epoch: 0, iteration: 200, Loss: 0.4382542371749878
Train Epoch: 0, iteration: 300, Loss: 0.11234814673662186
Train Epoch: 0, iteration: 400, Loss: 0.15356646478176117
Train Epoch: 0, iteration: 500, Loss: 0.566688597202301
Train Epoch: 0, iteration: 600, Loss: 0.19531209766864777
Train Epoch: 0, iteration: 700, Loss: 0.12632505595684052
Train Epoch: 0, iteration: 800, Loss: 0.18341219425201416
Train Epoch: 0, iteration: 900, Loss: 0.11926780641078949
Train Epoch: 0, iteration: 1000, Loss: 0.010400695726275444
Train Epoch: 0, iteration: 1100, Loss: 0.19254815578460693
Train Epoch: 0, iteration: 1200, Loss: 0.08206900209188461
Train Epoch: 0, iteration: 1300, Loss: 0.20620642602443695
Train Epoch: 0, iteration: 1400, Loss: 0.09066422283649445
Train Epoch: 0, iteration

NLL loss的定义

$\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_{y_n} x_{n,y_n}, \quad
        w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore_index}\}$

In [31]:
# fashion MNist dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data", train=True, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data", train=False, download=True,
           transform=transforms.Compose([
               transforms.ToTensor(),
               transforms.Normalize((0.1307,), (0.3081,))
           ])),
    batch_size=batch_size, shuffle=True, 
    num_workers=1, pin_memory=True
)

lr = 0.01
momentum  = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

num_epochs = 2
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
    
torch.save(model.state_dict(), "fashion_mnist_cnn.pt")


Extracting ./fashion_mnist_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./fashion_mnist_data/FashionMNIST/raw



Extracting ./fashion_mnist_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./fashion_mnist_data/FashionMNIST/raw



Extracting ./fashion_mnist_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./fashion_mnist_data/FashionMNIST/raw



Extracting ./fashion_mnist_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./fashion_mnist_data/FashionMNIST/raw
Processing...
Done!
Train Epoch: 0, iteration: 0, Loss: 2.282010078430176
Train Epoch: 0, iteration: 100, Loss: 1.6013178825378418
Train Epoch: 0, iteration: 200, Loss: 1.0022977590560913
Train Epoch: 0, iteration: 300, Loss: 0.4290510416030884
Train Epoch: 0, iteration: 400, Loss: 0.5959975719451904
Train Epoch: 0, iteration: 500, Loss: 0.34721431136131287
Train Epoch: 0, iteration: 600, Loss: 0.44426316022872925
Train Epoch: 0, iteration: 700, Loss: 0.6316989660263062
Train Epoch: 0, iteration: 800, Loss: 0.4500674605369568
Train Epoch: 0, iteration: 900, Loss: 0.3377816379070282
Train Epoch: 0, iteration: 1000, Loss: 0.2843500077724457
Train Epoch: 0, iteration: 1100, Loss: 0.48287028074264526
Train Epoch: 0, iteration: 1200, Loss: 0.43505892157554626
Train Epoch: 0, iteration: 1300, Loss: 0.4006580114364624
Train Epoch: 0, iteration: 1400, Loss: 0.41746336221694946
T